In [1]:
import json
import random
from pathlib import Path

import load_data
import numpy as np
import stylized_score

## Set seeds for reconstruction of the results

In [2]:
# Set seeds.
SEED = 12345
np.random.seed(SEED)
random.seed(SEED)

#### Load Data

Filter out selected times and stocks

In [3]:
min_len = 9216
B = 500
S = 8
L = 4096
price_df = load_data.load_prices("sp500")
log_ret, symbols = load_data.get_log_returns(price_df, min_len, True)
stf = stylized_score.boostrap_stylized_facts(log_ret, B, S, L)

Timespan 1987-12-17 -- 2024-07-17
Number of symbols 234


# get data
run_dir = Path('../data/garch_runs')
experiments = []
for i, path in enumerate(run_dir.iterdir()):
    if path.is_dir():
        print(f'Working on {str(path)}, nr {i}')

        with (path / 'meta_data.json').open() as file:
            config = json.load(file)

        if config['config']['vol'] == 'EGARCH':
            config['name'] = 'EGARCH'
        elif config['config']['vol'] == 'FIGARCH':
            config['name'] = 'FIGARCH'
        elif config['config']['vol'] == 'ARCH':
            config['name'] = 'ARCH'
        elif config['config']['vol'] == 'GARCH':
            if config['config']['power'] == 1:
                config['name'] = 'TARCH'
            elif 'o' in set(config['config'].keys()):
                config['name'] = 'GJR-GARCH'
            else:
                config['name'] = 'GARCH'
            


        garch_models = train_garch.load_garch(path / 'garch_models.pt')
        def garch_sampler(S):
            with warnings.catch_warnings(action="ignore"):
                return train_garch.sample_garch(garch_models, S, L)

        try:
            m_stf = stylized_score.stylied_facts_from_model(garch_sampler, B, S)
            tot, ind, _ = stylized_score.stylized_score(stf, m_stf)

            experiments.append({
                    "score": tot,
                    "ind_scores": ind,
                    "path": str(path),
                    "config": config['config'],
                    "name": config['name'],
                })
        except:  # noqa E722
            experiments.append({"score": 2**31, "config": config['config']})

        experiments.sort(key=lambda x: x["score"])
        with (run_dir / 'garch_experiments.json').open("w") as file:
                file.write(json.dumps(experiments, ensure_ascii=True, indent=4))

In [16]:
path = Path("../data/garch_runs")
with (path / "garch_experiments.json").open() as file:
    experiments = json.load(file)

best_experiments = {
    "GJR-GARCH": [],
    "GARCH": [],
    "TARCH": [],
    "EGARCH": [],
    "FIGARCH": [],
    "ARCH": [],
}
for exp in experiments:
    if np.isnan(exp["score"]):
        continue
    name = exp["name"]
    tot = exp["score"]
    sc = exp["ind_scores"]
    p = exp["config"]["p"]
    dist = exp["config"]["dist"]
    best_experiments[name].append((tot, sc, p, dist, name))

In [19]:
presented_experiments = []
for key in best_experiments.keys():
    best_experiments[key] = sorted(best_experiments[key], key=lambda x: x[0])
    for i in range(2):
        presented_experiments.append(best_experiments[key][i])

# Print Tables for thesis

In [20]:
table_start = r"""\begin{table}
  \centering
  \begin{threeparttable}
    \caption{GARCH Experiments}
    \label{tab:garch_exp}
        \begin{tabular}{c|c|c|l|l|l|l|l|l|l}
            \toprule
            Model & dist & p [[= o] = q] & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
            & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\"""
table_end = """            \\bottomrule
        \\end{tabular}
        \\begin{tablenotes}[flushleft]
            \\footnotesize
            \\item Stylized Scores for fitted GARCH distributions.
        \\end{tablenotes}
  \\end{threeparttable}
\\end{table}"""

print(table_start)
for mu, scs, p, dist, model in presented_experiments:
    ind_scores_str = " & ".join([f"${d:.2f}$" for d in scs])
    print(r"            \hline")
    print(f"            {model} & {dist} & {p} & {mu:.2f} & {ind_scores_str}\\\\")

print(table_end)

\begin{table}
  \centering
  \begin{threeparttable}
    \caption{GARCH Experiments}
    \label{tab:garch_exp}
        \begin{tabular}{c|c|c|l|l|l|l|l|l|l}
            \toprule
            Model & dist & p [[= o] = q] & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
            & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\
            \hline
            GJR-GARCH & skewt & 2 & 0.71 & $1.37$ & $0.65$ & $0.25$ & $0.62$ & $1.06$ & $0.29$\\
            \hline
            GJR-GARCH & skewt & 4 & 0.74 & $1.43$ & $0.69$ & $0.27$ & $0.64$ & $1.10$ & $0.31$\\
            \hline
            GARCH & t & 2 & 0.82 & $1.37$ & $0.66$ & $0.27$ & $1.27$ & $1.01$ & $0.33$\\
            \hline
            GARCH & skewt & 2 & 0.83 & $1.36$ & $0.66$ & $0.24$ & $1.39$ & $1.00$ & $0.34$\\
            \hline
            TARCH & t & 2 & 1.02 & $1.37$ & $0.38$ & $1.99$ & $0.53$ & $1.61$ & $0.25$\\
            \hline
            TARCH & t & 4 & 1.04 & $1.32$ & $0.3